In [4]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import jellyfish
import snowflake.connector
import time

import warnings
warnings.filterwarnings('ignore')


/home/rkothari/.local/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
cnx = snowflake.connector.connect(user='rkothari', password='Fall19@123', account='FF80747.east-us-2.azure', warehouse='DASCI_WH_S', database='FDC_STAGE', schema='DASCI', role='DASCI_DEVELOPER')


In [6]:
con = cnx.cursor()

In [30]:
# query = '''SELECT I.INVOICE_ITEM_FTS_ID ,INVOICE_UPC_CASE,  I.INVOICE_PACKAGE_DESCRIPTION ,p.GTIN, P.PRODUCT_MASTER_CATALOG_KEY , p.PRODUCT_DESCRIPTION
#    FROM FDC_PROD_LIVE.FDC02.ENTITY_INVOICE i JOIN PRODUCT_MASTER_CATALOG p 
#    on   cast(p.PRODUCT_MASTER_CATALOG_KEY as varchar) = i.INVOICE_UPC_CASE
# WHERE  i.INVOICE_DATE >= '2020-01-01' AND LEN(i.INVOICE_UPC_CASE) < 10
#    AND  i.INVOICE_PACKAGE_DESCRIPTION not in (SELECT INVOICE_PACKAGE_DESCRIPTION FROM INVOICE_NONPRODUCT_FILTER )
# '''

# query = '''
# SELECT DISTINCT PRODUCT_MASTER_CATALOG_KEY,PRODUCT_DESCRIPTION FROM PRODUCT_MASTER_CATALOG
#  '''

# query = '''SELECT DISTINCT PRODUCT_MASTER_CATALOG_KEY,PRODUCT_DESCRIPTION FROM PRODUCT_MASTER_CATALOG_MAPPED_TEMP
# '''

# query = '''SELECT DISTINCT PRODUCT_MASTER_CATALOG_KEY,PRODUCT_DESCRIPTION FROM PRODUCT_MASTER_CATALOG limit 10'''

query = """SELECT DISTINCT I.INVOICE_UPDATE_DATETIME,I.INVOICE_UPC_CASE,I.INVOICE_UPC_PACK,I.INVOICE_PACKAGE_DESCRIPTION , P.GTIN, P.INVOICE_UPC_CASE, P.PRODUCT_DESCRIPTION
FROM FDC_PROD_LIVE.FDC02.ENTITY_INVOICE I, 
FDC_STAGE.DASCI.RK_PROD_CAT_MAPPED_TEMP P
WHERE  INVOICE_UPDATE_DATETIME BETWEEN '2021-08-10 01:00:00.000' AND '2021-08-10 02:00:00.000' 
AND (I.INVOICE_UPC_CASE = P.INVOICE_UPC_CASE OR I.INVOICE_UPC_PACK = P.INVOICE_UPC_CASE )"""

# upc_less_than_10 = pd.read_sql(query, cnx, coerce_float = False)

In [31]:
# results = con.execute(query)
# res = pd.DataFrame.from_records(iter(results), columns=[y[0] for y in results.description])
# res.head()
# res.to_csv("~/Downloads/mapped_unique_masterkey_product_desc.csv")

In [32]:
result = pd.read_sql(query,cnx)

In [33]:
len(result)

82524

In [34]:
result.head(50)

,INVOICE_UPDATE_DATETIME,INVOICE_UPC_CASE,INVOICE_UPC_PACK,INVOICE_PACKAGE_DESCRIPTION,GTIN,INVOICE_UPC_CASE,PRODUCT_DESCRIPTION
0,2021-08-10 01:34:06,080660953199,080660953151,PACIFICO 12 PACK BOTTLE,00080660953199,080660953199,PACIFICO CLARA BEER 24 CT 12 OZ LONG NECK BTL ...
1,2021-08-10 01:38:23,018200967702,018200059896,MICHELOB ULTRA 24/12 NR BTL 4/6,00018200967702,018200967702,MICHELOB ULTRA BEER 6 CT 12 OZ LONG NECK BTL C...
2,2021-08-10 01:25:30,018200967702,018200059896,MICHELOB ULTRA 12OZ BOTTLE 4 X 6PK,00018200967702,018200967702,MICHELOB ULTRA BEER 6 CT 12 OZ LONG NECK BTL C...
3,2021-08-10 01:41:50,080660956299,080660956251,CORONA LT 4/6/12 NR,00080660956251,080660956251,CORONA LIGHT BEER 6 CT 12 OZ LONG NECK BTL CRT...
4,2021-08-10 01:55:25,,082242013741,BOTA BOX NGHTHAWK BLK RED BLND (3.L),00082242013741,082242013741,BOTA BOX NIGHTHAWK RED WINE BLEND 3/3L
5,2021-08-10 01:36:08,018200967702,018200059896,MICHELOB ULTRA 4/6/12 LN,00018200967702,018200967702,MICHELOB ULTRA BEER 6 CT 12 OZ LONG NECK BTL C...
6,2021-08-10 01:25:26,018200967702,018200059896,MICHELOB ULTRA 12OZ BOTTLE 4 X 6PK,00018200967702,018200967702,MICHELOB ULTRA BEER 6 CT 12 OZ LONG NECK BTL C...
7,2021-08-10 01:22:04,034100577070,034100577070,LITE 1/15/16 ALUM NR,00034100577070,034100577070,MILLER LITE BEER 15 CT 16 OZ ALUMINUM BOTTL IN...
8,2021-08-10 01:38:27,034100577070,034100577070,LITE ALMTK 15PK 16OZ CAN,00034100577070,034100577070,MILLER LITE BEER 15 CT 16 OZ ALUMINUM BOTTL IN...
9,2021-08-10 01:04:37,089924278986,089924278986,YUENGLING LAGER 2/12 PK CAN,00089924278986,089924278986,YUENGLING TRADITIONAL LAGER BEER 12 CT 12 OZ C...


In [35]:
result.to_csv("~/Downloads/Mapped_Catalog.csv")